In [1]:
import gzip
import json
import os
from document_preprocessor import RegexTokenizer
from indexing import Indexer, IndexType, BasicInvertedIndex
from ranker import *

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
DATA_PATH = 'data/'  # TODO: Set this to the path to your data folder
CACHE_PATH = '__pycache__/'  # Set this to the path of the cache folder

BEAUTY_PATH = DATA_PATH + 'meta_All_Beauty.jsonl.gz'
FASHION_PATH = DATA_PATH + 'meta_Amazon_Fashion.jsonl.gz'
COMBINE_PATH = DATA_PATH + 'Beauty_and_Fashion.jsonl.gz'
STOPWORD_PATH = DATA_PATH + 'stopwords.txt'
MAIN_INDEX = 'main_index'
TITLE_INDEX = 'title_index'
N_DOC_NEEDED = 50
DOCID_TO_TITLE_PATH = CACHE_PATH + 'docid_to_title.pkl'
DOCID_TO_LINK_PATH = CACHE_PATH + 'docid_to_link.pkl'

In [3]:
# Load stopwords
stopwords = set()
with open(STOPWORD_PATH, 'r') as f:
    for line in f:
        stopwords.add(line.strip())
        
print('Loaded', len(stopwords), 'stopwords.')

Loaded 543 stopwords.


In [12]:
# Load two categories' items into one dataset
item_cnt = 0
keys_to_keep = ["main_category", "title", "average_rating", "rating_number", "price", "images", "details"]

def process_dataset(input_path, output_file, item_cnt):
    with gzip.open(input_path, 'rt') as infile:
        for line in infile:
            data = json.loads(line)
            if data['description'] == [] and data['features'] == []:
                continue
            item_cnt += 1
            filtered_data = {key:data[key] for key in keys_to_keep if key in data}
            filtered_data['docid'] = item_cnt
            filtered_data['description'] = " ".join(data['features'] + data['description'])
            filtered_data['link'] = "https://www.amazon.com/dp/" + data['parent_asin']
            output_file.write(json.dumps(filtered_data) + '\n')
            
    return item_cnt

with gzip.open(COMBINE_PATH, 'wt') as outfile:
    item_cnt = process_dataset(BEAUTY_PATH, outfile, item_cnt)
    N_BEAUTY = item_cnt
    item_cnt = process_dataset(FASHION_PATH, outfile, item_cnt)
    N_FASHION = item_cnt - N_BEAUTY
    
print(f'Added {item_cnt} items in total to {COMBINE_PATH} from both Beauty and Fashion.')

Added 493293 items in total to data/Beauty_and_Fashion.jsonl.gz from both Beauty and Fashion.


In [27]:
print('Loading indexes...')
preprocessor = RegexTokenizer('\w+')
import time
start_time = time.time()
if not os.path.exists(MAIN_INDEX):
    main_index = Indexer.create_index(
        IndexType.BasicInvertedIndex, COMBINE_PATH, preprocessor,
        stopwords, 3, text_key='description', max_docs=493293
    )
    main_index.save(MAIN_INDEX)
else:
    main_index = BasicInvertedIndex()
    main_index.load(MAIN_INDEX)
print(time.time() - start_time)    


Loading indexes...
6212.378998041153


In [28]:
start_time = time.time()
if not os.path.exists(TITLE_INDEX):
    title_index = Indexer.create_index(
        IndexType.BasicInvertedIndex, COMBINE_PATH, preprocessor, 
        stopwords, 2, max_docs=493293,
        text_key='title'
    )
    title_index.save(TITLE_INDEX)
else:
    title_index = BasicInvertedIndex()
    title_index.load(TITLE_INDEX)
print(time.time() - start_time) 

19043.13917684555


In [32]:
import gzip
import pickle
import tqdm
if not os.path.exists(DOCID_TO_TITLE_PATH):
    docid_to_title = {}
    with gzip.open(COMBINE_PATH, mode = 'rt', newline = '') as f:
        for line in f:
            data = json.loads(line)
            docid_to_title[data['docid']] = data['title']
    pickle.dump(docid_to_title,
                open(DOCID_TO_TITLE_PATH, 'wb')
    )
else:
    docid_to_title = pickle.load(open(DOCID_TO_TITLE_PATH, 'rb'))

In [33]:
if not os.path.exists(DOCID_TO_LINK_PATH):
    docid_to_link = {}
    with gzip.open(COMBINE_PATH, mode = 'rt', newline = '') as f:
        for line in f:
            data = json.loads(line)
            docid_to_link[data['docid']] = data['link']
    pickle.dump(docid_to_link,
                open(DOCID_TO_LINK_PATH, 'wb')
    )
else:
    docid_to_link = pickle.load(open(DOCID_TO_LINK_PATH, 'rb'))

In [39]:
ranker = Ranker(main_index, preprocessor, stopwords, BM25(main_index))

import pandas as pd
beauty_queries = ["Hydrating face serum",
"Organic lip balm",
"Sunscreen spf 50",
"Matte foundation",
"Hair repair oil",
"Anti-aging night cream for sensitive skin",
"Cruelty-free makeup set",
"Gentle facial cleanser with natural ingredients",
"Long-lasting waterproof mascara",
"Shampoo and conditioner set for curly hair",
]
fashion_queries = ["Maxi dress",
"Crop top",
"V-neck t-shirt",
"Gray baggy jeans",
"Wool scarf",
"Running shoes with cushions",
"Lightweight travel backpack",
"High-waisted leggings with pockets",
"Casual blazer for men in slim fit style",
"Kids’ winter coat waterproof",
]

for beauty_query in beauty_queries:
    doc_lst = ranker.query(beauty_query)[:N_DOC_NEEDED]
    df = pd.DataFrame(columns=['query','title','docid','link','rel'])
    for i in range(len(doc_lst)):
        df.loc[i] = [beauty_query, docid_to_title[doc_lst[i][0]], doc_lst[i][0], docid_to_link[doc_lst[i][0]], None]
    df.to_csv(beauty_query+'.csv', index=False)

for fashion_query in fashion_queries:
    doc_lst = ranker.query(fashion_query)[:N_DOC_NEEDED]
    df = pd.DataFrame(columns=['query','title','docid','link','rel'])
    for i in range(len(doc_lst)):
        df.loc[i] = [fashion_query, docid_to_title[doc_lst[i][0]], doc_lst[i][0], docid_to_link[doc_lst[i][0]], None]
    df.to_csv(fashion_query+'.csv', index=False)